## Dataset Download

In [1]:
!gdown 1LQoyq_ZjHJih7hxJ2yJ_OrZhOIE_BZwB

Downloading...
From (original): https://drive.google.com/uc?id=1LQoyq_ZjHJih7hxJ2yJ_OrZhOIE_BZwB
From (redirected): https://drive.google.com/uc?id=1LQoyq_ZjHJih7hxJ2yJ_OrZhOIE_BZwB&confirm=t&uuid=757daf9e-d00f-4353-a9b8-806b25ac3855
To: /kaggle/working/depression_dataset_complete.zip
100%|█████████████████████████████████████████| 652M/652M [00:03<00:00, 203MB/s]


In [2]:
!unzip depression_dataset_complete.zip > /dev/null

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from tqdm import tqdm
import json
import math

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
LR = 2e-5
MAX_LEN = 512
FUSION_DIM = 768
LABELS = ["Lack of Interest", "Feeling Down", "Eating Disorder",
          "Sleeping Disorder", "Low Self-Esteem", "Concentration Problem", "Self-Harm"]
LABEL_MAP = {label: i for i, label in enumerate(LABELS)}
NUM_CLASSES = len(LABELS)

## Custom Dataset

In [4]:
class TextOnlyDepressionDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        ocr_text = sample["ocr_text"]
        figurative_reasoning = sample["figurative_reasoning"]
        combined_text = ocr_text + " [SEP] " + figurative_reasoning

        encoding = self.tokenizer(
            combined_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
        )

        labels = torch.zeros(NUM_CLASSES)
        for category in sample["meme_depressive_categories"]:
            labels[LABEL_MAP[category]] = 1.0

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": labels
        }

def custom_collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['label'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'label': labels,
    }

## Model Definition

In [5]:
class TextOnlyModel(nn.Module):
    def __init__(self, text_model_name="bert-base-uncased", num_classes=7):
        super(TextOnlyModel, self).__init__()

        self.text_encoder = AutoModel.from_pretrained(text_model_name)
        self.text_dim = self.text_encoder.config.hidden_size

        self.text_projections = nn.ModuleList([
            nn.Linear(self.text_dim, FUSION_DIM) for _ in range(3)
        ])

        self.expert_nets = nn.ModuleList([
            nn.Sequential(
                nn.Linear(FUSION_DIM, FUSION_DIM),
                nn.LayerNorm(FUSION_DIM),
                nn.GELU(),
                nn.Dropout(0.2)
            ) for _ in range(4)
        ])

        self.moe_gate = nn.Linear(FUSION_DIM, 4)

        self.classifier = nn.Sequential(
            nn.Linear(FUSION_DIM, FUSION_DIM),
            nn.LayerNorm(FUSION_DIM),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(FUSION_DIM, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        text_outputs = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        text_features = [
            text_outputs.hidden_states[-i][:, 0] for i in range(1, 4)
        ]
        text_features = [
            proj(feat) for proj, feat in zip(self.text_projections, text_features)
        ]

        combined_feature = sum(text_features) / len(text_features)

        expert_outputs = [expert(combined_feature) for expert in self.expert_nets]
        expert_gates = torch.nn.functional.softmax(self.moe_gate(combined_feature), dim=1)

        moe_output = torch.zeros_like(expert_outputs[0])
        for i, expert_out in enumerate(expert_outputs):
            moe_output += expert_out * expert_gates[:, i].unsqueeze(1)

        logits = self.classifier(moe_output)

        return logits

## Training Functions

In [7]:
def train_text_only_model(model, train_data, val_data, epochs, model_save_name):
    tokenizer = AutoTokenizer.from_pretrained("mental/mental-roberta-base")

    train_dataset = TextOnlyDepressionDataset(train_data, tokenizer, max_len=MAX_LEN)
    val_dataset = TextOnlyDepressionDataset(val_data, tokenizer, max_len=MAX_LEN)

    print("Train Set Size:", len(train_dataset))
    print("Validation Set Size:", len(val_dataset))

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=custom_collate_fn
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=custom_collate_fn
    )

    optimizer = optim.AdamW(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    criterion = nn.BCEWithLogitsLoss()

    model = model.to(DEVICE)
    model = nn.DataParallel(model)

    best_f1 = 0
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        model.train()
        train_loss = 0
        all_train_preds, all_train_labels = [], []

        for batch in tqdm(train_loader, desc="Training"):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()
            with torch.no_grad():
                predictions = (torch.sigmoid(logits) > 0.5).float().cpu().numpy()
                all_train_preds.extend(predictions)
                all_train_labels.extend(labels.cpu().numpy())

        train_loss = train_loss / len(train_loader)
        train_metrics = compute_multilabel_metrics(np.array(all_train_labels), np.array(all_train_preds))

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Macro-F1: {train_metrics['macro_f1']:.4f}, Weighted-F1: {train_metrics['weighted_f1']:.4f}")

        val_loss, val_metrics = evaluate_text_only_model(
            model, val_loader, criterion
        )

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro-F1: {val_metrics['macro_f1']:.4f}, Weighted-F1: {val_metrics['weighted_f1']:.4f}")

        scheduler.step(val_metrics['macro_f1'])
        
        f1_hm = 2 * val_metrics["macro_f1"] * val_metrics["weighted_f1"] / (val_metrics["macro_f1"] + val_metrics["weighted_f1"])
        if f1_hm > best_f1:
            best_f1 = f1_hm
            torch.save(model.state_dict(), f"{model_save_name}_depression.pth")
            print("Best model saved!")
        
    return model

def compute_multilabel_metrics(y_true, y_pred):
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    
    per_class_precision, per_class_recall, per_class_f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
    metrics = {
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'weighted_precision': weighted_precision,
        'weighted_recall': weighted_recall,
        'weighted_f1': weighted_f1,
        'per_class_precision': per_class_precision,
        'per_class_recall': per_class_recall,
        'per_class_f1': per_class_f1
    }
    
    return metrics

def evaluate_text_only_model(model, loader, criterion):
    model.eval()
    val_loss = 0
    all_val_preds, all_val_labels = [], []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE)
            
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            
            val_loss += loss.item()
            predictions = (torch.sigmoid(logits) > 0.5).float().cpu().numpy()
            all_val_preds.extend(predictions)
            all_val_labels.extend(labels.cpu().numpy())
    
    val_loss = val_loss / len(loader)
    val_metrics = compute_multilabel_metrics(np.array(all_val_labels), np.array(all_val_preds))
    
    print("\nPer-class metrics:")
    for i, label in enumerate(LABELS):
        print(f"{label}: F1={val_metrics['per_class_f1'][i]:.4f}, "
              f"Precision={val_metrics['per_class_precision'][i]:.4f}, "
              f"Recall={val_metrics['per_class_recall'][i]:.4f}")
    
    return val_loss, val_metrics

## Model Training

In [9]:
train_data = json.load(open("depression_train_llava_dataset.json", "r"))
val_data = json.load(open("depression_val_llava_dataset.json", "r"))
test_data = json.load(open("depression_test_llava_dataset.json", "r"))

tokenizer = AutoTokenizer.from_pretrained("mental/mental-roberta-base")

test_dataset = TextOnlyDepressionDataset(test_data, tokenizer, max_len=MAX_LEN)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=custom_collate_fn
)

In [ ]:
model = TextOnlyModel(
    text_model_name="mental/mental-roberta-base",
    num_classes=NUM_CLASSES
)

trained_model = train_text_only_model(
    model,
    train_data,
    val_data,
    epochs=30,
    model_save_name="only_text"
)

Train Set Size: 8722
Validation Set Size: 359


Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/30


Training: 100%|██████████| 546/546 [08:36<00:00,  1.06it/s]


Train Loss: 0.2759
Train Macro-F1: 0.5342, Weighted-F1: 0.5673


...


Epoch 30/30


Training: 100%|██████████| 546/546 [08:38<00:00,  1.05it/s]


Train Loss: 0.0036
Train Macro-F1: 0.9962, Weighted-F1: 0.9961


Evaluating: 100%|██████████| 23/23 [00:06<00:00,  3.50it/s]



Per-class metrics:
Lack of Interest: F1=0.3548, Precision=0.6471, Recall=0.2444
Feeling Down: F1=0.6565, Precision=0.8060, Recall=0.5538
Eating Disorder: F1=0.7723, Precision=0.7500, Recall=0.7959
Sleeping Disorder: F1=0.6875, Precision=0.6471, Recall=0.7333
Low Self-Esteem: F1=0.3200, Precision=0.5000, Recall=0.2353
Concentration Problem: F1=0.7222, Precision=0.8667, Recall=0.6190
Self-Harm: F1=0.5593, Precision=0.5789, Recall=0.5410
Validation Loss: 0.8549
Validation Macro-F1: 0.5818, Weighted-F1: 0.5832


## Inference

In [10]:
!gdown 1f7yxLZbZm_2O4dJVs4Zpok6XTl_Q94ka

Downloading...
From (original): https://drive.google.com/uc?id=1f7yxLZbZm_2O4dJVs4Zpok6XTl_Q94ka
From (redirected): https://drive.google.com/uc?id=1f7yxLZbZm_2O4dJVs4Zpok6XTl_Q94ka&confirm=t&uuid=0817ade8-3841-4cca-a9ff-def0f3d3cca6
To: /kaggle/working/only_text_depression.pth
100%|████████████████████████████████████████| 518M/518M [00:10<00:00, 51.1MB/s]


In [11]:
def inference(test_data, model_path):
    model = TextOnlyModel(
        text_model_name="mental/mental-roberta-base",
        num_classes=NUM_CLASSES
    )
    
    model = model.to(DEVICE)

    weights = torch.load(model_path, map_location=DEVICE, weights_only=True)
    weights_single = {k.replace("module.", ""): v for k, v in weights.items()}

    model.load_state_dict(weights_single)
    tokenizer = AutoTokenizer.from_pretrained("mental/mental-roberta-base")

    test_dataset = TextOnlyDepressionDataset(test_data, tokenizer, max_len=MAX_LEN)
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=custom_collate_fn
    )

    loss, metrics = evaluate_text_only_model(
        model, test_loader, nn.BCEWithLogitsLoss()
    )

    print(f"Test Loss: {loss:.4f}")
    print(f"Test Macro-F1: {metrics['macro_f1']:.4f}, Weighted-F1: {metrics['weighted_f1']:.4f}")

In [13]:
inference(test_data, "only_text_depression.pth")

Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating: 100%|██████████| 33/33 [00:13<00:00,  2.42it/s]


Per-class metrics:
Lack of Interest: F1=0.4800, Precision=0.8276, Recall=0.3380
Feeling Down: F1=0.7244, Precision=0.7195, Recall=0.7294
Eating Disorder: F1=0.8208, Precision=0.8765, Recall=0.7717
Sleeping Disorder: F1=0.6950, Precision=0.7903, Recall=0.6203
Low Self-Esteem: F1=0.4479, Precision=0.5513, Recall=0.3772
Concentration Problem: F1=0.7759, Precision=0.9000, Recall=0.6818
Self-Harm: F1=0.5672, Precision=0.7170, Recall=0.4691
Test Loss: 0.4901
Test Macro-F1: 0.6445, Weighted-F1: 0.6527
